# Introdução

# Materiais e Métodos:

## Dependências

In [20]:
%pip install kagglehub
%pip install pandas
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import random
import seaborn as sns
import numpy as np
import psutil
import os

## DataSet

### Importando o DataSet

In [45]:
books = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Books.csv", sep=";", header=0)
users = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Users.csv", sep=";", low_memory=False, header=0)
ratings = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Ratings.csv", sep=";", low_memory=False, header=0)

### Visualização dos dados importados

In [46]:
print("Books")
print(books.head(), "\n")

print("Informações sobre os livros")
print(books.info(), "\n")

Books
         ISBN                                              Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 Author  Year                Publisher  
0    Mark P. O. Morford  2002  Oxford University Press  
1  Richard Bruce Wright  2001    HarperFlamingo Canada  
2          Carlo D'Este  1991          HarperPerennial  
3      Gina Bari Kolata  1999     Farrar Straus Giroux  
4       E. J. W. Barber  1999   W. W. Norton & Company   

Informações sobre os livros
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISBN       271379 

In [47]:
print("Ratings")
print(ratings.head(), "\n")

print("Informações sobre os ratings")
print(ratings.info(), "\n")



Ratings
   User-ID        ISBN  Rating
0   276725  034545104X       0
1   276726  0155061224       5
2   276727  0446520802       0
3   276729  052165615X       3
4   276729  0521795028       6 

Informações sobre os ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   User-ID  1149780 non-null  int64 
 1   ISBN     1149780 non-null  object
 2   Rating   1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None 



In [48]:
print("Users")
print(users.head(), "\n")

print("Informações sobre os usuários")
print(users.info(), "\n")

Users
  User-ID  Age
0       1  NaN
1       2   18
2       3  NaN
3       4   17
4       5  NaN 

Informações sobre os usuários
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278859 entries, 0 to 278858
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   User-ID  278859 non-null  object
 1   Age      168627 non-null  object
dtypes: object(2)
memory usage: 4.3+ MB
None 



### Ajustando os dados

In [49]:
# Books
books.dropna(subset=["Title", "Author", "Publisher"], inplace=True)
books["ISBN"] = books["ISBN"].str.strip()
books["Title"] = books["Title"].str.strip()
books["Author"] = books["Author"].str.strip()
books["Publisher"] = books["Publisher"].str.strip()
books.drop_duplicates(subset=["ISBN"], inplace=True)

books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271374 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISBN       271374 non-null  object
 1   Title      271374 non-null  object
 2   Author     271374 non-null  object
 3   Year       271374 non-null  int64 
 4   Publisher  271374 non-null  object
dtypes: int64(1), object(4)
memory usage: 12.4+ MB


In [52]:
# Ratings
ratings.dropna(inplace=True)
ratings["User-ID"] = ratings["User-ID"].astype(str).str.strip()
ratings["ISBN"] = ratings["ISBN"].str.strip()
ratings = ratings[(ratings["Rating"] >= 0) & (ratings["Rating"] <= 10)]
ratings = ratings.groupby(["User-ID", "ISBN"], as_index=False)["Rating"].mean()
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   User-ID  1149780 non-null  object 
 1   ISBN     1149780 non-null  object 
 2   Rating   1149780 non-null  float64
dtypes: float64(1), object(2)
memory usage: 26.3+ MB


In [58]:
# Users
users["Age"] = pd.to_numeric(users["Age"], errors="coerce")
users = users[(users["Age"] >= 0) & (users["Age"] <= 100)]
users.drop_duplicates(inplace=True)

/tmp/ipykernel_29433/3679153470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.drop_duplicates(inplace=True)


# Implementação do Algoritmo

# Avaliação do Algoritmo